In [7]:
from pycoral.adapters import classify, common
from pycoral.utils.edgetpu import make_interpreter
import platform
import tflite_runtime.interpreter as tflite

_EDGETPU_SHARED_LIB = {
  'Linux': 'libedgetpu.so.1',
  'Darwin': 'libedgetpu.1.dylib',
  'Windows': 'edgetpu.dll'
}[platform.system()]

print(_EDGETPU_SHARED_LIB)

interp = tflite.load_delegate(_EDGETPU_SHARED_LIB)

print(help(interp))

edgetpu.dll
Help on Delegate in module tflite_runtime.interpreter object:

class Delegate(builtins.object)
 |  Delegate(library, options=None)
 |  
 |  Python wrapper class to manage TfLiteDelegate objects.
 |  
 |  The shared library is expected to have two functions:
 |    TfLiteDelegate* tflite_plugin_create_delegate(
 |        char**, char**, size_t, void (*report_error)(const char *))
 |    void tflite_plugin_destroy_delegate(TfLiteDelegate*)
 |  
 |  The first one creates a delegate object. It may return NULL to indicate an
 |  error (with a suitable error message reported by calling report_error()).
 |  The second one destroys delegate object and must be called for every
 |  created delegate object. Passing NULL as argument value is allowed, i.e.
 |  
 |    tflite_plugin_destroy_delegate(tflite_plugin_create_delegate(...))
 |  
 |  always works.
 |  
 |  Methods defined here:
 |  
 |  __del__(self)
 |  
 |  __init__(self, library, options=None)
 |      Loads delegate from the sh

In [7]:
from sklearn.datasets import fetch_openml
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
import pandas as pd
import os


def get_mnist784_dataset():
    data_dir = os.path.join(os.getcwd(), "datasets")
    if not os.path.exists(data_dir):
        os.mkdir(data_dir)
    mnist_dir =  os.path.join(os.getcwd(), "datasets", "mnist_784")
    if not os.path.exists(mnist_dir):
        os.mkdir(mnist_dir)

    mnist = fetch_openml('mnist_784', as_frame=False)
    data_df = pd.DataFrame(mnist.data, columns=mnist.feature_names)
    target_df = pd.DataFrame(mnist.target, columns=mnist.target_names)
    data_path = os.path.join(os.getcwd(), "datasets", "mnist_784", "mnist_784_data.csv")
    target_path = os.path.join(os.getcwd(), "datasets", "mnist_784", "mnist_784_target.csv")
    data_df.to_csv(data_path)
    target_df.to_csv(target_path)
    X, y = mnist.data, mnist.target
    return X, y

def mnist784_df_from_csv():
    data_path = os.path.join(os.getcwd(), "datasets", "mnist_784", "mnist_784_data.csv")
    target_path = os.path.join(os.getcwd(), "datasets", "mnist_784", "mnist_784_target.csv")
    data_df = pd.read_csv(data_path)
    target_df = pd.read_csv(target_path)
    X_df, y_df = data_df, target_df
    X_df.drop(columns="Unnamed: 0", inplace=True)
    y_df.drop(columns="Unnamed: 0", inplace=True)
    return X_df, y_df

def train_predict_mnist784():
    X_df, y_df = mnist784_df_from_csv()
    X = X_df.values
    y = y_df.values
    some_digit = X[0]
    X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
    y_train_5 = (y_train == 5)
    y_test_5 = (y_test == 5)
    sgd_clf = SGDClassifier(random_state=42)
    sgd_clf.fit(X_train, y_train_5)
    sgd_clf.predict([some_digit])
    scores = cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")
    return scores

res = train_predict_mnist784()
print(res)

c:\py_repos\image_classifier\venv\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\py_repos\image_classifier\venv\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\py_repos\image_classifier\venv\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\py_repos\image_classifier\venv\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please 

[0.95035 0.96035 0.9604 ]
